In [1]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt

cwd = os.getcwd()
fileLoc = 'dataFiles/pls_200points_MRT_torqueAdded.csv'
# fileLoc = 'dataFiles/test_data1_WithOutputs_withCalculatedValues.csv'
dataFile_original = pd.read_csv(os.path.join(cwd,fileLoc))
# print(dataFile_original.head())

In [2]:
dataFile_incomplete = dataFile_original.drop(['Year','Experiments', 'Sr No','Screw Configuration','Liq add position','DetMRT','RanTorque','DetTorque','RanMRT','Regime'],axis=1)
# dataFile_incomplete = dataFile_original.drop(['Experiments', 'Sr No','Screw Configuration','Liq add position','DetMRT','RanTorque','DetTorque','RanMRT','Regime','Beta','Exp Fill level','Fines %'],axis=1)
# print(dataFile_incomplete.describe)

In [3]:
from HelperTools import DataCompletionMethods
dcm_object = DataCompletionMethods.DataCompletionMethods(dataFile_incomplete)
# detMRT = dataFile_incomplete['DetMRT']
# ranMRT = dataFile_incomplete['RanMRT']
dataFile_DettorqueMRT = dcm_object.lineaeRegressionModel(['Torque','MRT'],True)
dataFile_incomplete[['Det_Torque','Det_MRT']] = dataFile_DettorqueMRT
dataFile_original[['Det_Torque','Det_MRT']] = dataFile_DettorqueMRT
dataFile_RantorqueMRT = dcm_object.randomImputationRegression(['Torque','MRT'],True)
dataFile_incomplete[['Ran_Torque','Ran_MRT']] = dataFile_RantorqueMRT
dataFile_original[['Ran_Torque','Ran_MRT']] = dataFile_RantorqueMRT
ranMRT = dataFile_incomplete['Ran_MRT']
FillLevel = dcm_object.fillLevel_osorio(ranMRT)
dataFile_complete = dataFile_incomplete
# dataFile_complete.to_csv('newDataCompleted.csv')
# dataFile_complete["Calc fill level"] = FillLevel
# print(dataFile_complete["Calc fill level"].describe())
# FillLevel_lalith = dcm_object.fillevel_lalith(detMRT)
# dataFile_complete["Lal Calc fill level"] = FillLevel_lalith
# print(dataFile_complete["Lal Calc fill level"].describe())

KeyError: "None of [Index(['DetTorque', 'DetMRT'], dtype='object')] are in the [columns]"

In [ ]:
from sklearn.cross_decomposition  import PLSRegression, PLSCanonical
dataFile_complete = dataFile_complete.drop(['Torque','MRT'],axis=1)
train_dataset = dataFile_complete.sample(frac=0.8,random_state=42)
exp_traindataset = dataFile_original['Experiments'].sample(frac=0.8,random_state=42)
test_dataset = dataFile_complete.drop(train_dataset.index)
exp_testdataset = dataFile_original['Experiments'].drop(train_dataset.index)

In [ ]:
labels = ['DetTorque','RanMRT','final d50']
train_labels = pd.DataFrame([train_dataset.pop(i) for i in labels]).T
test_labels = pd.DataFrame([test_dataset.pop(i) for i in labels]).T
print(train_labels.describe())

In [ ]:
pls1 = PLSRegression(n_components=2,scale=True)
pls1.fit(train_dataset,train_labels)
print(np.round(pls1.coef_,3))
pls1.predict(test_dataset)
pls1.score(test_dataset,test_labels)

In [ ]:
ad = pls1.transform(train_dataset)
extent = np.divide(train_labels['final d50'],train_dataset['Initial d50'])
exps = exp_traindataset
fig,ax = plt.subplots(1,2,subplot_kw=dict(box_aspect=1),
    sharey=True)
for g in np.unique(exps):
    i = np.where(exps==g)
    ax[0].scatter(ad[i,0],ad[i,1],label=g)
ax[0].set_xlabel('LV 1')
ax[0].set_ylabel('LV 2')
ax[0].set_title('According to exp.')
ax[0].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ranges = [0,15,20,25,30,35,50]
train_dataset_pls = train_dataset.copy()
train_dataset_pls['Extent of gran'] = extent
train_dataset_pls['LV 1'] = ad[:,0]
train_dataset_pls['LV 2'] = ad[:,1]
groups = train_dataset_pls.groupby(pd.cut(train_dataset_pls['Extent of gran'],ranges))
for val, group in groups:
    ax[1].scatter(group['LV 1'],group['LV 2'],label=val)
ax[1].set_xlabel('LV 1')
ax[1].set_ylabel('LV 2')
ax[1].set_xlim([-5,5])
ax[1].set_ylim([-5,5])
plt.title('Extent of granulation')
fig.set_size_inches(10,5)
fig.suptitle('PLS latent variables',fontsize=16)
ax[1].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0)

In [ ]:
# Tsne 
from sklearn.manifold import TSNE
from sklearn import preprocessing
train_dataset_minmax = preprocessing.MinMaxScaler().fit_transform(train_dataset)
tsne = TSNE(n_components=2, random_state=42)
train_dataset_transform = tsne.fit_transform(train_dataset_minmax)


In [ ]:
fig,ax = plt.subplots(1,2,subplot_kw=dict(box_aspect=1),
    sharey=True)
for g in np.unique(exps):
    i = np.where(exps==g)
    ax[0].scatter(train_dataset_transform[i,0],train_dataset_transform[i,1],label=g)
ax[0].set_xlabel('LV 1')
ax[0].set_ylabel('LV 2')
ax[0].set_title('According to exp.')
ax[0].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ranges = [0,15,20,25,30,35,50]
train_dataset_tsne = train_dataset.copy()
train_dataset_tsne['Extent of gran'] = extent
train_dataset_tsne['LV 1'] = train_dataset_transform[:,0]
train_dataset_tsne['LV 2'] = train_dataset_transform[:,1]
groups = train_dataset_tsne.groupby(pd.cut(train_dataset_tsne['Extent of gran'],ranges))
for val, group in groups:
    ax[1].scatter(group['LV 1'],group['LV 2'],label=val)
ax[1].set_xlabel('LV 1')
ax[1].set_ylabel('LV 2')
plt.title('Extent of granulation')
fig.set_size_inches(10,5)
fig.suptitle('TSNE latent variables',fontsize=16)
ax[1].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0)

In [ ]:
#PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
prinComps = pca.fit_transform(train_dataset_minmax)


In [ ]:
fig,ax = plt.subplots(1,2,subplot_kw=dict(box_aspect=1),
    sharey=True)
for g in np.unique(exps):
    i = np.where(exps==g)
    ax[0].scatter(prinComps[i,0],prinComps[i,1],label=g)
ax[0].set_xlabel('LV 1')
ax[0].set_ylabel('LV 2')
ax[0].set_title('According to exp.')
ax[0].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ranges = [0,15,20,25,30,35,50]
train_dataset_pca = train_dataset.copy()
train_dataset_pca['Extent of gran'] = extent
train_dataset_pca['LV 1'] = prinComps[:,0]
train_dataset_pca['LV 2'] = prinComps[:,1]
groups = train_dataset_pca.groupby(pd.cut(train_dataset_pca['Extent of gran'],ranges))
for val, group in groups:
    ax[1].scatter(group['LV 1'],group['LV 2'],label=val)
ax[1].set_xlabel('LV 1')
ax[1].set_ylabel('LV 2')
plt.title('Extent of granulation')
fig.set_size_inches(10,5)
fig.suptitle('PCA latent variables',fontsize=16)
ax[1].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0)

In [ ]:
# autoencoder
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers
import seaborn as sns

#### linear encoder
#input layer

input_layer = Input(shape=(train_dataset_minmax.shape[1],))

# encoding using only 1 linear layer
encoder1 = Dense(2,activation='linear')(input_layer)

# decoder 0 layers and output 
output1 = Dense(train_dataset_minmax.shape[1],activation='linear')(encoder1)

autoencoder = Model(input_layer,output1)
autoencoder.compile(optimizer='adadelta',loss='mse')
print(autoencoder.summary())
autoencoder.fit(train_dataset_minmax,train_dataset_minmax,epochs=1,shuffle=False,validation_split=0.25,verbose=0)

In [ ]:
#getting latent variable values 
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])

latent_rep = np.array(hidden_representation.predict(train_dataset_minmax))
fig,ax = plt.subplots(1,2,subplot_kw=dict(box_aspect=1),
    sharey=True)
for g in np.unique(exps):
    i = np.where(exps==g)
    ax[0].scatter(latent_rep[i,0],latent_rep[i,1],label=g)
ax[0].set_xlabel('LV 1')
ax[0].set_ylabel('LV 2')
ax[0].set_title('According to exp.')
ax[0].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ranges = [0,15,20,25,30,35,50]
train_dataset_aeliner = train_dataset.copy()
train_dataset_aeliner['Extent of gran'] = extent
train_dataset_aeliner['LV 1'] = latent_rep[:,0]
train_dataset_aeliner['LV 2'] = latent_rep[:,1]
groups = train_dataset_aeliner.groupby(pd.cut(train_dataset_aeliner['Extent of gran'],ranges))
for val, group in groups:
    ax[1].scatter(group['LV 1'],group['LV 2'],label=val)
ax[1].set_xlabel('LV 1')
ax[1].set_ylabel('LV 2')
plt.title('Extent of granulation')
fig.set_size_inches(10,5)
fig.suptitle('Linear AE latent variables',fontsize=16)
ax[1].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0)

In [ ]:
#### linear 2 layer encoder
#input layer

input_layer = Input(shape=(train_dataset_minmax.shape[1],))

# encoding using only 1 linear layer
encoder2 = Dense(4,activation='linear')(input_layer)
encoder3 = Dense(2,activation='linear')(encoder2)

# decoder 0 layers and output 
decoder1 = Dense(4,activation='linear')(encoder3)
output2 = Dense(train_dataset_minmax.shape[1],activation='linear')(decoder1)

autoencoder2 = Model(input_layer,output2)
autoencoder2.compile(optimizer='adadelta',loss='mse')
print(autoencoder2.summary())
autoencoder2.fit(train_dataset_minmax,train_dataset_minmax,epochs=1,shuffle=True,validation_split=0.25)
#getting latent variable values 
hidden_representation2 = Sequential()
hidden_representation2.add(autoencoder2.layers[0])
hidden_representation2.add(autoencoder2.layers[1])
hidden_representation2.add(autoencoder2.layers[2])

latent_rep2 = np.array(hidden_representation2.predict(train_dataset_minmax))
fig,ax = plt.subplots(1,2,subplot_kw=dict(box_aspect=1),
    sharey=True)
for g in np.unique(exps):
    i = np.where(exps==g)
    ax[0].scatter(latent_rep2[i,0],latent_rep2[i,1],label=g)
ax[0].set_xlabel('LV 1')
ax[0].set_ylabel('LV 2')
ax[0].set_title('According to exp.')
ax[0].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ranges = [0,15,20,25,30,35,50]
train_dataset_aeliner2 = train_dataset.copy()
train_dataset_aeliner2['Extent of gran'] = extent
train_dataset_aeliner2['LV 1'] = latent_rep2[:,0]
train_dataset_aeliner2['LV 2'] = latent_rep2[:,1]
groups = train_dataset_aeliner2.groupby(pd.cut(train_dataset_aeliner2['Extent of gran'],ranges))
for val, group in groups:
    ax[1].scatter(group['LV 1'],group['LV 2'],label=val)
ax[1].set_xlabel('LV 1')
ax[1].set_ylabel('LV 2')
plt.title('Extent of granulation')
fig.set_size_inches(10,5)
fig.suptitle('Linear AE 2 latent variables',fontsize=16)
ax[1].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0)

In [ ]:
#### non-linear 1 layer encoder
#input layer

input_layer = Input(shape=(train_dataset_minmax.shape[1],))

# encoding using only 1 linear layer
encoder4 = Dense(3,activation='tanh')(input_layer)


# decoder 0 layers and output 

output3 = Dense(train_dataset_minmax.shape[1],activation='tanh')(encoder4)

autoencoder3 = Model(input_layer,output3)
autoencoder3.compile(optimizer='adadelta',loss='mse')
print(autoencoder3.summary())
autoencoder3.fit(train_dataset_minmax,train_dataset_minmax,epochs=1,shuffle=True,validation_split=0.25)
#getting latent variable values 
hidden_representation3 = Sequential()
hidden_representation3.add(autoencoder3.layers[0])
hidden_representation3.add(autoencoder3.layers[1])


latent_rep3 = np.array(hidden_representation3.predict(train_dataset_minmax))
fig,ax = plt.subplots(1,2,subplot_kw=dict(box_aspect=1),
    sharey=True)
for g in np.unique(exps):
    i = np.where(exps==g)
    ax[0].scatter(latent_rep3[i,0],latent_rep3[i,1],label=g)
ax[0].set_xlabel('LV 1')
ax[0].set_ylabel('LV 2')
ax[0].set_title('According to exp.')
ax[0].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ranges = [0,15,20,25,30,35,50]
train_dataset_aenonlin1 = train_dataset.copy()
train_dataset_aenonlin1['Extent of gran'] = extent
train_dataset_aenonlin1['LV 1'] = latent_rep3[:,0]
train_dataset_aenonlin1['LV 2'] = latent_rep3[:,1]
groups = train_dataset_aenonlin1.groupby(pd.cut(train_dataset_aenonlin1['Extent of gran'],ranges))
for val, group in groups:
    ax[1].scatter(group['LV 1'],group['LV 2'],label=val)
ax[1].set_xlabel('LV 1')
ax[1].set_ylabel('LV 2')
plt.title('Extent of granulation')
fig.set_size_inches(10,5)
fig.suptitle('non-Linear AE 1 latent variables',fontsize=16)
ax[1].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0)

In [ ]:
#### non-linear 2 layer encoder
#input layer

input_layer = Input(shape=(train_dataset_minmax.shape[1],))

# encoding using only 1 linear layer
encoder5 = Dense(3,activation='tanh')(input_layer)
encoder6 = Dense(2,activation='linear')(encoder5)

# decoder 0 layers and output 
decoder2 = Dense(3,activation='tanh')(encoder6)
output4 = Dense(train_dataset_minmax.shape[1],activation='tanh')(decoder2)

autoencoder4 = Model(input_layer,output4)
autoencoder4.compile(optimizer='adadelta',loss='mse')
print(autoencoder4.summary())
autoencoder4.fit(train_dataset_minmax,train_dataset_minmax,epochs=1,shuffle=True,validation_split=0.25,verbose=0)
#getting latent variable values 
hidden_representation4 = Sequential()
hidden_representation4.add(autoencoder4.layers[0])
hidden_representation4.add(autoencoder4.layers[1])
hidden_representation4.add(autoencoder4.layers[2])

latent_rep4 = np.array(hidden_representation4.predict(train_dataset_minmax))
fig,ax = plt.subplots(1,3,sharey=True,subplot_kw=dict(box_aspect=1.25))
for g in np.unique(exps):
    i = np.where(exps==g)
    ax[0].scatter(latent_rep4[i,0],latent_rep4[i,1],label=g)
ax[0].set_xlabel('LV 1')
ax[0].set_ylabel('LV 2')
ax[0].set_title('According to exp.')
ax[0].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ranges = [0,15,20,25,30,35,50]
train_dataset_aenonlin2 = train_dataset.copy()
train_dataset_aenonlin2['Extent of gran'] = extent
train_dataset_aenonlin2['LV 1'] = latent_rep4[:,0]
train_dataset_aenonlin2['LV 2'] = latent_rep4[:,1]
groups = train_dataset_aenonlin2.groupby(pd.cut(train_dataset_aenonlin2['Extent of gran'],ranges))
for val, group in groups:
    ax[1].scatter(group['LV 1'],group['LV 2'],label=val)
ax[1].set_xlabel('LV 1')
ax[1].set_ylabel('LV 2')
ax[2].set_title('Extent of granulation')
fig.set_size_inches(15,5)
fig.suptitle('non-Linear AE 2 latent variables',fontsize=16)
ax[1].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0)
ranges = [0,0.25,0.5,0.75,1]
train_dataset_aenonlin2['Calc Fill level'] = dcm_object.fillLevel_osorio(train_dataset_aenonlin2['DetMRT'])
groups = train_dataset_aenonlin2.groupby(pd.cut(train_dataset_aenonlin2['Calc Fill level'],ranges))
for val, group in groups:
    ax[2].scatter(group['LV 1'],group['LV 2'],label=val)
ax[2].set_xlabel('LV 1')
ax[2].set_ylabel('LV 2')
ax[2].set_title('Fill level')
ax[2].legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)

In [ ]:
from sklearn.cluster import KMeans
from numpy import unique, where

kmeans_clus = KMeans(n_clusters=5)
kmeans_clus.fit(latent_rep4)
y_kmeans = kmeans_clus.predict(latent_rep4)
clusters = unique(y_kmeans)
print(kmeans_clus.cluster_centers_)
for cluster in clusters:
	# get row indexes for samples with this cluster
	row_ix = where(y_kmeans == cluster)
	# create scatter of these samples
	plt.scatter(latent_rep4[row_ix, 0], latent_rep4[row_ix, 1], label=cluster)
plt.legend()
plt.xlabel('LV 1')
plt.ylabel('LV 2')

In [ ]:
#### non-linear 2 layer encoder supervised learning
#input layer

train_labels['final d50'] = train_labels['final d50'] / 1e6
train_labels['DetTorque'] = train_labels['DetTorque'] / 10
train_labels['RanMRT'] = train_labels['RanMRT'] / 100

input_layer = Input(shape=(train_dataset_minmax.shape[1],))

# encoding using only 1 linear layer
encoder7 = Dense(3,activation='tanh')(input_layer)
encoder8 = Dense(2,activation='linear')(encoder7)

# decoder 0 layers and output 
decoder3 = Dense(3,activation='tanh')(encoder8)
output5 = Dense(train_dataset_minmax.shape[1],activation='tanh')(decoder3)

decoder_exp1 = Dense(3,activation='tanh')(decoder3)
decoder_exp2 = Dense(3,activation='tanh')(decoder_exp1)

output_exp1 = Dense(train_labels.shape[1],activation='tanh')(decoder_exp2)
autoencoder_sup = Model(input_layer,[output5,output_exp1])
autoencoder_sup.compile(optimizer='adadelta',loss='mse')
print(autoencoder_sup.summary())
autoencoder_sup.fit(train_dataset_minmax,[train_dataset_minmax,train_labels],epochs=100,shuffle=True,validation_split=0.25,verbose=0)
#getting latent variable values 
hidden_representation5 = Sequential()
hidden_representation5.add(autoencoder_sup.layers[0])
hidden_representation5.add(autoencoder_sup.layers[1])
hidden_representation5.add(autoencoder_sup.layers[2])
latent_rep5 = np.array(hidden_representation5.predict(train_dataset_minmax))
fig,ax = plt.subplots(1,3,sharey=True)
for g in np.unique(exps):
    i = np.where(exps==g)
    ax[0].scatter(latent_rep5[i,0],latent_rep5[i,1],label=g)
ax[0].set_xlabel('LV 1')
ax[0].set_ylabel('LV 2')
ax[0].set_title('According to exp.')
ax[0].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ranges = [0,15,20,25,30,35,50]
train_dataset_aenonlin_sup = train_dataset.copy()
train_dataset_aenonlin_sup['Extent of gran'] = extent
train_dataset_aenonlin_sup['LV 1'] = latent_rep5[:,0]
train_dataset_aenonlin_sup['LV 2'] = latent_rep5[:,1]
groups = train_dataset_aenonlin_sup.groupby(pd.cut(train_dataset_aenonlin_sup['Extent of gran'],ranges))
for val, group in groups:
    ax[1].scatter(group['LV 1'],group['LV 2'],label=val)
ax[1].set_xlabel('LV 1')
ax[1].set_ylabel('LV 2')
ax[1].set_title('Extent of granulation')
fig.set_size_inches(15,5)
fig.suptitle('non-Linear AE 2 latent variables',fontsize=16)
ax[1].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0)
ranges = [0,0.25,0.5,0.75,1]
train_dataset_aenonlin_sup['Calc Fill level'] = dcm_object.fillLevel_osorio(train_dataset_aenonlin_sup['DetMRT'])
groups = train_dataset_aenonlin_sup.groupby(pd.cut(train_dataset_aenonlin_sup['Calc Fill level'],ranges))
for val, group in groups:
    ax[2].scatter(group['LV 1'],group['LV 2'],label=val)
ax[2].set_xlabel('LV 1')
ax[2].set_ylabel('LV 2')
ax[2].set_title('Fill level')
ax[2].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)

In [ ]:
kmeans_clus_sup = KMeans(n_clusters=4,algorithm="elkan")
kmeans_clus_sup.fit(latent_rep4)
y_kmeans_sup = kmeans_clus_sup.predict(latent_rep5)
clusters_sup = unique(y_kmeans_sup)
sup_center = kmeans_clus_sup.cluster_centers_
print(sup_center)
for cluster_sup in clusters_sup:
	# get row indexes for samples with this cluster
	row_ix = where(y_kmeans_sup == cluster_sup)
	# create scatter of these samples
	plt.scatter(latent_rep5[row_ix, 0], latent_rep5[row_ix, 1], label=cluster_sup)
plt.legend()
plt.scatter(sup_center[:,0],sup_center[:,1],c='black',s=200,alpha=0.5)
plt.xlabel('LV 1')
plt.ylabel('LV 2')

In [ ]:
datafile_processparam = dataFile_original[['RPM','L/S Ratio','FlowRate (kg/hr)', 'Temperature']]
datafile_material = dataFile_original[['Initial d50','Binder Viscosity (mPa.s)','Flowability (HR)','Bulk Density']]
datafile_geometry = dataFile_original[['nCE','Granulator diameter (mm)','L/D Ratio','SA of KE','nKE','Liq add position','nKZ','dKZ']]
datafile_allinputs = dataFile_original[['RPM','L/S Ratio','FlowRate (kg/hr)', 'Temperature','Initial d50','Binder Viscosity (mPa.s)','Flowability (HR)','Bulk Density','nCE','Granulator diameter (mm)','L/D Ratio','SA of KE','nKE','Liq add position','nKZ','dKZ']]
labels = dataFile_original[['DetTorque','RanMRT','final d50']]

datafile_material = preprocessing.MinMaxScaler().fit_transform(datafile_material)
datafile_processparam = preprocessing.MinMaxScaler().fit_transform(datafile_processparam)


labels['final d50'] = labels['final d50'] / 1e3
labels['DetTorque'] = labels['DetTorque'] / 10
labels['RanMRT'] = labels['RanMRT'] / 100

##LV for Process Parameters
input_pp = Input(shape=(datafile_processparam.shape[1],))
#hidden layer
hidden_pp1 = Dense(2,activation='tanh',name='hid_layer_pp1')(input_pp)
# bottleneck layer
encoder_pp = Dense(1,activation='tanh',name='lv_layer_pp')(hidden_pp1)


##LV for Material Properties
input_mat = Input(shape=(datafile_material.shape[1],))
#hidden layer
hidden_mat = Dense(2,activation='tanh',name='hid_layer_mat')(input_mat)
# bottleneck layer
encoder_mat = Dense(1,activation='tanh',name='lv_layer_mat')(hidden_mat)

## label encoding for liq add position

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
datafile_geometry['Liq add position'] = le.fit_transform(datafile_geometry['Liq add position'])
datafile_geometry = preprocessing.MinMaxScaler().fit_transform(datafile_geometry)
datafile_allinputs['Liq add position'] = le.fit_transform(datafile_allinputs['Liq add position'])
datafile_allinputs_copy = datafile_allinputs.copy()
datafile_allinputs = preprocessing.MinMaxScaler().fit_transform(datafile_allinputs)

##LV for Geometry
input_geo = Input(shape=(datafile_geometry.shape[1],))
#hidden layer
hidden_geo = Dense(4,activation='tanh',name='hid_layer_geo')(input_geo)
# bottleneck layer
encoder_geo = Dense(1,activation='tanh',name='lv_layer_geo')(hidden_geo)

from keras.layers import Concatenate
concat_bottleneck = Concatenate()([encoder_pp,encoder_mat,encoder_geo])
# recons decoder layer 1
decoder_recons = Dense(6,activation='tanh',name='decoder_layer_recons1')(concat_bottleneck)

# output prediction decoder layer 1 

decoder_pred = Dense(3,activation='tanh',name='decoder_layer_pred1')(concat_bottleneck)

# output layer reconstruction
output_recons = Dense(datafile_allinputs.shape[1],activation='tanh',name='recon_out')(decoder_recons)
output_pred = Dense(train_labels.shape[1],activation='linear',name='pred_out')(decoder_pred)

autoencoder = Model([input_pp,input_mat,input_geo],[output_recons,output_pred])
autoencoder.compile(optimizer='Adam',loss='mse',metrics=['mse','mae'])

# if fitting is reqd 
autoencoder.fit([datafile_processparam,datafile_material,datafile_geometry],[datafile_allinputs,labels],epochs=100, shuffle=True,validation_split=0.25,verbose=1)

# autoencoder.summary()

predictions_y_AE = autoencoder.predict([datafile_processparam,datafile_material,datafile_geometry])

# Getting the hidden representation of the 
hidden_representation_3 = Model([input_pp,input_mat,input_geo],[encoder_pp,encoder_mat,encoder_geo])
# hidden_representation_3.add(Concatenate()([autoencoder.layers[0],autoencoder.layers[1],autoencoder.layers[2]]))

# hidden_representation_3.summary()
latent_rep = np.array(hidden_representation_3.predict([datafile_processparam,datafile_material,datafile_geometry]))
# hidden_representation_3.summary()

In [ ]:
pred_con_y_AE = np.ravel(np.array(predictions_y_AE[1]))
pred_con_y_AE = np.reshape(pred_con_y_AE,(len(datafile_processparam),3))
con_pre_y_pd = pd.DataFrame({'Pre final d50':pred_con_y_AE[:,2], 'Pre Torque':pred_con_y_AE[:,0], 'Pre MRT':pred_con_y_AE[:,1]})
con_pre_y_pd_fin = pd.concat([pd.DataFrame(datafile_allinputs),con_pre_y_pd],axis=1,ignore_index=False)
con_pre_y_pd_fin.to_csv('AE_prediction.csv')

In [ ]:
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'qt')

fig = plt.figure(figsize=(8,8))
ax = plt.axes(projection='3d')
# print(latent_rep.shape)
len_TrainData = len(train_dataset)
# print(len_TrainData)
for g in np.unique(exps):
    i = np.where(exps==g)
    ax.scatter3D(latent_rep[0,i],latent_rep[1,i],latent_rep[2,i],label=g)
ax.set_xlabel('PP')
ax.set_ylabel('Mat.')
ax.set_zlabel('Geo')
ax.set_title('According to exp.')
ax.legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ax.view_init(45, 30)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = plt.axes(projection='3d')
# print(latent_rep.shape)
regs = dataFile_original['Regime']
# print(len_TrainData)
for g in np.unique(regs):
    i = np.where(regs==g)
    ax.scatter3D(latent_rep[0,i],latent_rep[1,i],latent_rep[2,i],label=g)
ax.set_xlabel('PP')
ax.set_ylabel('Mat.')
ax.set_zlabel('Geo')
ax.set_title('According to Regime')
ax.legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ax.view_init(-45, 30)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = plt.axes(projection='3d')
# print(latent_rep.shape)
regs = dataFile_original['Granulator diameter (mm)']
# print(len_TrainData)
for g in np.unique(regs):
    i = np.where(regs==g)
    ax.scatter3D(latent_rep[0,i],latent_rep[1,i],latent_rep[2,i],label=g)
ax.set_xlabel('PP')
ax.set_ylabel('Mat.')
ax.set_zlabel('Geo')
ax.set_title('According to Granulator size')
ax.legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ax.view_init(-45, 30)


In [ ]:

fig = plt.figure(figsize=(8,8))
ax = plt.axes(projection='3d')
# print(latent_rep.shape)
regs = dataFile_original['nKE']
# print(len_TrainData)
for g in np.unique(regs):
    i = np.where(regs==g)
    ax.scatter3D(latent_rep[0,i],latent_rep[1,i],latent_rep[2,i],label=g)
ax.set_xlabel('PP')
ax.set_ylabel('Mat.')
ax.set_zlabel('Geo')
ax.set_title('According to nKE')
ax.legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ax.view_init(-45, 30)


In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')
datafile_allinputs_copy['PP'] = latent_rep[0,:]
datafile_allinputs_copy['Mat'] = latent_rep[1,:]
datafile_allinputs_copy['geo'] = latent_rep[2,:]
fig,ax = plt.subplots(1,3,subplot_kw=dict(box_aspect=1),
    sharey=True)
for g in np.unique(exps):
    i = np.where(exps==g)
    ax[0].scatter(latent_rep[0,i],latent_rep[1,i],label=g)
ax[0].set_xlabel('PP')
ax[0].set_ylabel('Mat')
ax[0].set_title('According to exp.')
ax[0].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ranges = [0,15,20,25,30,35,50]
datafile_allinputs_copy['Extent of gran'] = labels['final d50'] / datafile_allinputs_copy['Initial d50'] /1e-6
groups = datafile_allinputs_copy.groupby(pd.cut(datafile_allinputs_copy['Extent of gran'],ranges))
for val, group in groups:
    ax[1].scatter(group['PP'],group['Mat'],label=val)
ax[1].set_xlabel('PP')
ax[1].set_ylabel('Mat')
plt.title('Extent of granulation')
fig.set_size_inches(15,5)
fig.suptitle('AE latent representation',fontsize=16)
ax[1].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0)
regs = dataFile_original['Regime']
for g in np.unique(regs):
    i = np.where(regs==g)
    ax[2].scatter(latent_rep[0,i],latent_rep[1,i],label=g)
ax[2].set_xlabel('PP')
ax[2].set_ylabel('Mat')
ax[2].set_title('According to regimes')
ax[2].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)

In [ ]:

fig,ax = plt.subplots(1,3,subplot_kw=dict(box_aspect=1),
    sharey=True)
for g in np.unique(exps):
    i = np.where(exps==g)
    ax[0].scatter(latent_rep[1,i],latent_rep[2,i],label=g)
ax[0].set_xlabel('Mat')
ax[0].set_ylabel('geo')
ax[0].set_title('According to exp.')
ax[0].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ranges = [0,15,20,25,30,35,50]
datafile_allinputs_copy['Extent of gran'] = labels['final d50'] / datafile_allinputs_copy['Initial d50'] /1e-6
groups = datafile_allinputs_copy.groupby(pd.cut(datafile_allinputs_copy['Extent of gran'],ranges))
for val, group in groups:
    ax[1].scatter(group['Mat'],group['geo'],label=val)
ax[1].set_xlabel('Mat')
ax[1].set_ylabel('geo')
plt.title('Extent of granulation')
fig.set_size_inches(15,5)
fig.suptitle('AE latent representation',fontsize=16)
ax[1].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0)
for g in np.unique(regs):
    i = np.where(regs==g)
    ax[2].scatter(latent_rep[1,i],latent_rep[2,i],label=g)
ax[2].set_xlabel('Mat')
ax[2].set_ylabel('geo')
ax[2].set_title('According to regimes')
ax[2].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)

In [ ]:
fig,ax = plt.subplots(1,3,subplot_kw=dict(box_aspect=1),
    sharey=True)
for g in np.unique(exps):
    i = np.where(exps==g)
    ax[0].scatter(latent_rep[0,i],latent_rep[2,i],label=g)
ax[0].set_xlabel('PP')
ax[0].set_ylabel('geo')
ax[0].set_title('According to exp.')
ax[0].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)
ranges = [0,15,20,25,30,35,50]
datafile_allinputs_copy['Extent of gran'] = labels['final d50'] / datafile_allinputs_copy['Initial d50'] /1e-6
groups = datafile_allinputs_copy.groupby(pd.cut(datafile_allinputs_copy['Extent of gran'],ranges))
for val, group in groups:
    ax[1].scatter(group['PP'],group['geo'],label=val)
ax[1].set_xlabel('PP')
ax[1].set_ylabel('geo')
plt.title('Extent of granulation')
fig.set_size_inches(15,5)
fig.suptitle('AE latent representation',fontsize=16)
ax[1].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=3, mode="expand", borderaxespad=0)
for g in np.unique(regs):
    i = np.where(regs==g)
    ax[2].scatter(latent_rep[0,i],latent_rep[2,i],label=g)
ax[2].set_xlabel('PP')
ax[2].set_ylabel('geo')
ax[2].set_title('According to regimes')
ax[2].legend(bbox_to_anchor=(0., -0.5, 1., .102), loc='lower left',
           ncol=2, mode="expand", borderaxespad=0.)